# Coeffs for Oskar

In [11]:
# imports
from importlib import reload

import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime, time
#import random 

import pandas

from oceancolor.tara import io as tara_io
from oceancolor.utils import cat_utils

from cnmf.oceanography import iops
from cnmf import io as cnmf_io

# Load me

## Loisel

In [4]:
nmf_fit = 'L23'
Ncomp = 3
d = cnmf_io.load_nmf(nmf_fit, Ncomp, 'a')

Loading: /home/xavier/Projects/Oceanography/python/cnmf/cnmf/data/L23/L23_NMF_03_a.npz


In [6]:
coeff = d['coeff']
coeff.shape

(3320, 3)

## Tara

In [7]:
nmf_fit = 'Tara'
N_NMF = 3
d_tara = cnmf_io.load_nmf(nmf_fit, N_NMF, 'a')

Loading: /home/xavier/Projects/Oceanography/python/cnmf/cnmf/data/Tara/Tara_NMF_03_a.npz


In [8]:
d_tara['UID']

array([1609011540000000000, 1609011600000000000, 1609011660000000000, ...,
       1657176120000000000, 1657176180000000000, 1657176240000000000])

# Load Tara db

In [10]:
tara_df = tara_io.load_pg_db()

Reading: /home/xavier/Projects/Oceanography/python/ocean-color/oceancolor/data/Tara/merged_tara_pacific_microbiome_acs_160124.feather


/home/xavier/Projects/Oceanography/python/ocean-color/oceancolor/tara/io.py:83: UserWarning: Duplicate times in Tara Oceans database
  warnings.warn("Duplicate times in Tara Oceans database")


Using bit_flags removes 127034 rows of a total 774298


In [14]:
tara_df.head()

,dt,lat,lon,t,s,ad_model400,agaus406,agaus434,agaus453,agaus470,...,hbb_bbp_25,hbb_bbp_26,hbb_bbp_27,hbb_bbp_28,fdom_sd,fdom_sensor,geometry,uid,mission_id,passes_flags
utc_dt,,,,,,,,,,,,,,,,,,,,,
2016-05-30 07:05:00,30-May-2016 07:05:00,45.673750,-8.814967,15.5537,35.5332,0.000000,0.007691,0.005692,0.004222,0.003370,...,NaN,NaN,NaN,NaN,NaN,None,b'\x01\x01\x00\x00\x00K\xd9\x9bOC\xa1!\xc0\xe1...,1464591900000000000,0,True
2016-05-30 07:06:00,30-May-2016 07:06:00,45.672600,-8.816917,15.5490,35.5332,0.000817,0.006569,0.005500,0.003743,0.003245,...,NaN,NaN,NaN,NaN,NaN,None,b'\x01\x01\x00\x00\x00 b\xbf\xe6B\xa2!\xc0\xeb...,1464591960000000000,0,True
2016-05-30 07:07:00,30-May-2016 07:07:00,45.671500,-8.819000,15.5370,35.5330,0.000000,0.008018,0.005824,0.004188,0.003485,...,NaN,NaN,NaN,NaN,NaN,None,b'\x01\x01\x00\x00\x00$y\xd2\xf7S\xa3!\xc0;)E\...,1464592020000000000,0,True
2016-05-30 07:08:00,30-May-2016 07:08:00,45.670400,-8.821117,15.5385,35.5330,0.000000,0.008432,0.006026,0.004189,0.003642,...,NaN,NaN,NaN,NaN,NaN,None,b'\x01\x01\x00\x00\x00X\x9a]gi\xa4!\xc0*Q\xcd\...,1464592080000000000,0,True
2016-05-30 07:09:00,30-May-2016 07:09:00,45.669383,-8.823250,15.5483,35.5329,0.000000,0.007812,0.006014,0.004180,0.003538,...,NaN,NaN,NaN,NaN,NaN,None,b'\x01\x01\x00\x00\x00/\xdd$\x06\x81\xa5!\xc0\...,1464592140000000000,0,True


# Match up

In [13]:
mid = cat_utils.match_ids(d_tara['UID'], tara_df.uid.values, require_in_match=True)

# New table

In [21]:
df = pandas.DataFrame()
# meta
df['uid'] = tara_df.uid.values[mid]
df['lat'] = tara_df.lat.values[mid]
df['lon'] = tara_df.lon.values[mid]
df['t'] = tara_df.t.values[mid]
df['s'] = tara_df.s.values[mid]

# Coefficients
df['H1'] = d_tara['coeff'][:,0]
df['H2'] = d_tara['coeff'][:,1]
df['H3'] = d_tara['coeff'][:,2]
#

## Write

In [22]:
df.to_csv('tara_coeffs_for_oskar.csv', index=False)